In [1]:
import csv
import os
#os.system('conda install numpy')
import numpy
import string
from collections import defaultdict
#os.system('conda install nltk')
import nltk

In [2]:
import spacy
from __future__ import unicode_literals

In [3]:
print("Reading data..." )    
        
with open('questions.csv', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    keys = next(reader)
    data = [row for row in reader]  
    
print("done")

Reading data...
done


In [4]:
#indexes for values in data
pid = 0
qid1 = 1
qid2 = 2
q1 = 3
q2 = 4
dup = 5

In [5]:
qPairs = []
for t in data:
    t[5] = int(t[5])
    qPairs.append(t[3:])

In [6]:
dupPairs = []
nonPairs = []
for q in qPairs:
    if q[2] == 0:
        nonPairs.append(q)
    else:
        dupPairs.append(q)

In [7]:
nonPairs[534]

['Does my life hold any significance to the world/universe?',
 'Do we truly have any purpose in life? Or do we create a purpose to make ourselves feel significant in the very vast world, or to make ourselves feel that our existence in the vast world is required?',
 0]

In [8]:
nonPairs[534][1:3]

['Do we truly have any purpose in life? Or do we create a purpose to make ourselves feel significant in the very vast world, or to make ourselves feel that our existence in the vast world is required?',
 0]

In [9]:
allQuestions = defaultdict(list)

In [10]:
dupPairs[0]

['Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
 "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?",
 1]

In [11]:
allQuestions = defaultdict(list)
for d in data:
    allQuestions[int(d[qid1])].append([int(d[qid2]),int(d[dup])])
    allQuestions[int(d[qid2])].append([int(d[qid1]),int(d[dup])])

In [12]:
indices = []
for key in allQuestions:
    if len(allQuestions[key]) > 1:
        indices.append(key)

In [13]:
allQuestions[902]

[[901, 1], [623627, 0], [698850, 0]]

In [14]:
print(dupPairs[0][0].lower())
print(dupPairs[0][1].lower())

astrology: i am a capricorn sun cap moon and cap rising...what does that say about me?
i'm a triple capricorn (sun, moon and ascendant in capricorn) what does this say about me?


In [15]:
text1 = nltk.word_tokenize(dupPairs[0][0].lower())
text2 = nltk.word_tokenize(dupPairs[0][1].lower())
pos1 = nltk.pos_tag(text1)
pos2 = nltk.pos_tag(text2)
test = nltk.pos_tag(nltk.word_tokenize(qPairs[0][0].lower()))

In [16]:
print(pos1)
print(pos2)

[('astrology', 'NN'), (':', ':'), ('i', 'NN'), ('am', 'VBP'), ('a', 'DT'), ('capricorn', 'JJ'), ('sun', 'NN'), ('cap', 'NN'), ('moon', 'NN'), ('and', 'CC'), ('cap', 'NN'), ('rising', 'NN'), ('...', ':'), ('what', 'WP'), ('does', 'VBZ'), ('that', 'DT'), ('say', 'VBP'), ('about', 'IN'), ('me', 'PRP'), ('?', '.')]
[('i', 'NN'), ("'m", 'VBP'), ('a', 'DT'), ('triple', 'JJ'), ('capricorn', 'NN'), ('(', '('), ('sun', 'NN'), (',', ','), ('moon', 'NN'), ('and', 'CC'), ('ascendant', 'NN'), ('in', 'IN'), ('capricorn', 'NN'), (')', ')'), ('what', 'WP'), ('does', 'VBZ'), ('this', 'DT'), ('say', 'VB'), ('about', 'IN'), ('me', 'PRP'), ('?', '.')]


In [30]:
nouns1 = []
nouns2 = []
for pair in pos1:
    if pair[1] == 'NN':
        nouns1.append(pair[0])
for pair in pos2:
    if pair[1] == 'NN':
        nouns2.append(pair[0])

In [94]:
def toNouns(sent):
    nouns = {}
    pos = nltk.pos_tag(nltk.word_tokenize(sent.lower()))
    for pair in pos:
        if pair[1].startswith('NN'):
            if pair[0] in nouns:
                nouns[pair[0]] += 1
            else:
                nouns[pair[0]] = 1
    return nouns

In [95]:
def toVerbs(sent):
    verbs = {}
    pos = nltk.pos_tag(nltk.word_tokenize(sent.lower()))
    for pair in pos:
        if pair[1].startswith('VB'):
            if pair[0] in verbs:
                verbs[pair[0]] += 1
            else:
                verbs[pair[0]] = 1
    return verbs

In [28]:
print(nouns1)
print(nouns2)

['astrology', 'i', 'sun', 'cap', 'moon', 'cap', 'rising']
['i', 'capricorn', 'sun', 'moon', 'ascendant', 'capricorn']


In [20]:
def vectorize_string(s):
    '''
    turns a string into a dictionary of (substrings of len p). 
    Can be used as a 'vector' where noexistant keys have val 0
    
    Each (substrings of len p) maps to the amount of appearances
    of itself in the original string
    '''
    
    v = {}
    for word in s:
        if word in v:
            v[word] += 1
        else:
            v[word] = 1
            
    return v

In [21]:
def magnitude_dict(data):
    '''
    Evaulats the magnitude of a dict
    '''
    
    return sum([value**2 for value in data.values()])**.5


In [22]:
def dot_dicts(d1,d2):
    '''
    multiplies the values of the overlaping keys
    in two dictionaries.  It chooses to iterate
    over the smaller and lookup in the larger
    '''
    
    a = d1 if len(d1) < len(d2) else d2
    b = d2 if len(d1) < len(d2) else d1
    
    dot = 0
    # a is shorter than b
    for s in a:
        if s in b:
            dot += a[s]*b[s]
            
    return dot

In [23]:
def cosine_similarity_string(s1,s2):
    '''
    returns the cosine simularity of two strings
    words in each string are considered dimensions
    '''
    
    d1 = vectorize_string(s1)
    d2 = vectorize_string(s2)
    return dot_dicts(d1,d2)/(magnitude_dict(d1)*magnitude_dict(d2))

In [24]:
def cosine_sim_dict(d1,d2):
    return dot_dicts(d1,d2)/(magnitude_dict(d1)*magnitude_dict(d2))

In [50]:
cosine_sim_dict(toNouns(qPairs[4][0]), toNouns(qPairs[4][1]))

0.40824829046386296

In [116]:
z = 22
print(cosine_sim_dict(toNouns(qPairs[z][0]), toNouns(qPairs[z][1])))
print(cosine_sim_dict(toVerbs(qPairs[z][0]), toVerbs(qPairs[z][1])))
print(toNouns(qPairs[z][0]))
print(toNouns(qPairs[z][1]))
print(toVerbs(qPairs[z][0]))
print(toVerbs(qPairs[z][1]))
print(qPairs[z])
#print(qPairs[z][2])

0.4999999999999999
0.4999999999999999
{'questions': 1, 'quora': 1}
{'question': 1, 'quora': 1}
{'ask': 1, 'are': 1}
{'ask': 1, 'i': 1}
['What are the questions should not ask on Quora?', 'Which question should I ask on Quora?', 0]


In [45]:
qPairs[2]

['How can I increase the speed of my internet connection while using a VPN?',
 'How can Internet speed be increased by hacking through DNS?',
 0]

In [31]:
toNouns(data[3][q1])

{}

In [ ]:
data_no_empty = [d for d in data if len(toNouns(d[q1])) != 0 and len(toNouns(d[q2])) != 0]

In [ ]:
cs_all = [cosine_simularity_string(toNouns(d[q1]), toNouns(d[q2])) for d in data_no_empty]

In [ ]:
plt.title('Histogram of Cosine Simularity of pairs')
plt.xlabel('cosine simularity')
plt.ylabel('count')
bins = np.arange(0,1.01,.01)
plt.hist(cs_all, bins=bins)
plt.show()

In [35]:
sent = data[3][q2]
nltk.pos_tag(nltk.word_tokenize(sent.lower()))

[('find', 'VB'),
 ('the', 'DT'),
 ('remainder', 'NN'),
 ('when', 'WRB'),
 ('[', 'NNP'),
 ('math', 'NN'),
 (']', 'VBD'),
 ('23^', 'CD'),
 ('{', '('),
 ('24', 'CD'),
 ('}', ')'),
 ('[', 'NNP'),
 ('/math', 'NNP'),
 (']', 'NNP'),
 ('is', 'VBZ'),
 ('divided', 'VBN'),
 ('by', 'IN'),
 ('24,23', 'CD'),
 ('?', '.')]

In [34]:
len(data)

404351